# $M/HYPO_r/1$ queue
Arrival rate $\lambda$

perpareTransfer and sendTrytes service rates are $\mu_1$ and $\mu_2$, respectively.

$Q = \begin{pmatrix}
B_{00} &B_{01}  & 0 & 0 & 0 & \ldots\\ 
B_{10} & A_{1} & A_{2} & 0 & 0 &\dots \\ 
 0& A_{0} &  A_{1} & A_{2}  & 0 &\ldots \\ 
 0&  0& A_{0} &  A_{1}  & A_{2} & \\ 
 0&  0&  0& A_{0} &  A_{1}  & \ldots\\ 
 \vdots& \vdots & \vdots & \vdots & \vdots & \ddots
\end{pmatrix}$

- $A_0$: Transitions from state (k, r) to state (k − 1, 1), the transit rate of the last service phase.
- $A_1$: The superdiagonal elements of the matrices $A_1$ represent phase completion to state (k + 1, i ). It follows that the only nonzero elements in this matrix are the diagonal elements The diagonal elements are set equal to the negated sum of the off-diagonal elements of Q. All other at rate rμ in service phase i < r at level k > 0, i.e., transitions from state (k, i ) to state (k, i + 1).
- $A_2$: The matrices A2 represent arrivals at rate λ which can occur during service at any phase i at any level k > 0, i.e., transitions from state (k, i ) which are all equal to λ.
- $B_{01}$: A 1 × r row matrix all of whose elements are zero except for the first which is equal to $\lambda$. This is the rate of transition from state (0, 0) to state (1, 1) and corresponds to an arrival to an empty system.
- $B_{10}$: A r × 1 column matrix all of whose elements are zero except for the last which is equal to $\mu_r$. This is the rate of transition from state (1, r) to state (0, 0) and corresponds to the complete service termination of the only customer in the system.
- $B_{00}$: A 1×1 matrix whose nonzero element is $-\lambda$ and reflects the fact that the sum across the first row must be zero.

$A_0=\begin{pmatrix}
0 & 0  & 0 & 0 & \ldots & 0\\ 
0 & 0 & 0 & \ldots & \ldots &0 \\ 
 0& 0 &  0 & 0  & \ldots &0 \\ 
 0&  0& 0 &  0  & \ldots & 0\\ 
 \vdots& \vdots & \vdots & \vdots &\ddots & \vdots\\ 
 \mu_r & 0 & 0 & 0 & \ldots & 0
\end{pmatrix}$,$A_2=\lambda I_r$, $A_1 = \begin{pmatrix}
-\lambda-\mu_1 & \mu_1  & 0 & 0 & \ldots & 0\\ 
0 & -\lambda-\mu_2 & \mu_2 & 0 & \ldots &0 \\ 
0 & 0 &  -\lambda-\mu_3 & \mu_3  & \ldots &0 \\ 
\vdots& \vdots & \vdots & \ddots &\ddots & \vdots\\ 
0&  0& 0 &  0  & \vdots & \mu_{r-1}\\
 0 & 0 & 0 & 0 & \ldots & -\lambda-\mu_r
\end{pmatrix}$

$B_{00}=-\lambda$,$B_{01}=\begin{pmatrix}
\lambda & 0 & 0 & \ldots & 0
\end{pmatrix}$,$B_{10}=\begin{pmatrix}
0\\ 
0\\  
\vdots\\ 
0 \\
\mu_r
\end{pmatrix}$


In [217]:
import numpy as np
from numpy import kron,eye,zeros,ones,multiply,dot
from numpy.linalg import inv,norm,matrix_power

In [161]:
# M/HYPO2/1
## M Arrival Process:
lambda1 = 1
T = [-lambda1]
T0 = [lambda1]
xi = [1]

## E_2 Service Process:
mu1=8
mu2=4
S = [[-mu1, mu1],
     [0, -mu2]]
S0 = [[0],[mu2]]
sigma = [1,0]

## Block Submatrices for all types of queues:
ra = len(T)
rs = len(S[0])
A0 = kron(eye(ra), multiply(S0,sigma))
A1 = kron(T, eye(rs)) + kron(eye(ra), S)
A2 = kron(multiply(T0,xi), eye(rs))
B00 = T
B01 = kron(multiply(T0,xi),sigma)
B10 = kron(eye(ra),S0)
l = len(B00)
r = len(A0[0])

print("A0:",A0,'\n',"A1:",A1,'\n',"A2:",A2,'\n',"B00:",B00,'\n',"B01:",B01,'\n',"B10:",B10)

# Form Neuts’ R matrix  
V = dot(A2, inv(A1))
W = dot(A0, inv(A1))
R = zeros((r,r)) 
Rbis = -V - dot(dot(R,R), W)
n = 1
err = 1.0e-10
while (norm(R-Rbis,1) > err):
    R = Rbis
    Rbis = -V - dot(dot(R,R), W)
    n = n+1
R = Rbis

A0: [[0. 0.]
 [4. 0.]] 
 A1: [[-9.  8.]
 [ 0. -5.]] 
 A2: [[1. 0.]
 [0. 1.]] 
 B00: [-1] 
 B01: [1 0] 
 B10: [[0.]
 [4.]]


In [153]:
R

array([[0.15625, 0.25   ],
       [0.03125, 0.25   ]])

In [154]:
n

32

In [155]:
# check stability
meanLambda = lambda1
meanMu = 1/norm(-(sigma * inv(S)),1)
rho = meanLambda/meanMu
rho

0.125

In [221]:
# Solve boundary equations
head = np.concatenate((np.array(B00),B01),axis=0)
tail = np.concatenate((B10,A1+dot(R,A0)),axis=1)
N = np.vstack((head,tail))

N[0,r] = 1
for k in range(1,r+1):
    N[k,r]=0

rhs = zeros(r+1)
rhs[r] = 1

soln = dot(rhs,inv(N))
pi0 = soln[:1]
pi1 = soln[1:]
print(pi0,pi1)

e = ones((r,1))
sumation = norm(pi0,1) + dot(dot(pi1, inv(eye(r)-R)), e)
pi0 = pi0/sumation
pi1 = pi1/sumation
print(pi0,pi1)

# Print results
max = 10 # maximum population requested 
pop = zeros((max+1,1))
pop[0,0] = norm(pi0,1)
for k in range(1,max+1):
    pi = pi1 * matrix_power(R,(k-1)) # Get successive components of pi
    pop[k,0] = norm(pi,1)
print(pop)

# Measures of Effectiveness
EN = norm(pi1*matrix_power(inv(eye(r)-R),2),1)
ER = EN/meanLambda
print(EN,ER)

[1.] [0.15625 0.25   ]
[0.625] [0.09765625 0.15625   ]
[[6.25000000e-01]
 [1.56250000e-01]
 [7.81250000e-02]
 [2.68554687e-02]
 [8.46862793e-03]
 [2.60114670e-03]
 [7.92071223e-04]
 [2.40493100e-04]
 [7.29480961e-05]
 [2.21197547e-05]
 [6.70652233e-06]]
0.44726562494828637 0.44726562494828637


# Example 12.1

In [149]:
## M Arrival Process:
lambda1 = 1
T = [-lambda1]
T0 = [lambda1]
xi = [1]

## E_2 Service Process:
r = 3
mu = 1.5
mu1,mu2,mu3 = r*mu, r*mu, r*mu

S = [[-mu1, mu1,0],
     [0, -mu2,mu2],
    [0,0,-mu3]]
S0 = [[0],[0],[mu3]]
sigma = [1,0,0]

## Block Submatrices for all types of queues:
ra = len(T)
rs = len(S[0])
A0 = kron(eye(ra), multiply(S0,sigma))
A1 = kron(T, eye(rs)) + kron(eye(ra), S)
A2 = kron(multiply(T0,xi), eye(rs))
B00 = T
B01 = kron(multiply(T0,xi),sigma)
B10 = kron(eye(ra),S0)
l = len(B00)
r = len(A0[0])

print("A0:",A0,'\n',"A1:",A1,'\n',"A2:",A2,'\n',"B00:",B00,'\n',"B01:",B01,'\n',"B10:",B10)

# Form Neuts’ R matrix  
V = dot(A2, inv(A1))
W = dot(A0, inv(A1))
R = np.zeros((r,r)) 
Rbis = -V - dot(dot(R,R), W)
n = 1
err = 1.0e-10
while (norm(R-Rbis,1) > err):
    R = Rbis
    Rbis = -V - dot(dot(R,R), W)
    n = n+1
R = Rbis

A0: [[0.  0.  0. ]
 [0.  0.  0. ]
 [4.5 0.  0. ]] 
 A1: [[-5.5  4.5  0. ]
 [ 0.  -5.5  4.5]
 [ 0.   0.  -5.5]] 
 A2: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 
 B00: [-1] 
 B01: [1 0 0] 
 B10: [[0. ]
 [0. ]
 [4.5]]


In [150]:
n

73

In [151]:
R

array([[0.33196159, 0.27160494, 0.22222222],
       [0.10973937, 0.27160494, 0.22222222],
       [0.06035665, 0.04938272, 0.22222222]])